In [1]:
# --- CÀI ĐẶT MÔI TRƯỜNG VÀ IMPORTS ---

# Xóa thư mục cũ để đảm bảo phiên bản mới nhất
!rm -rf Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update

# Tải repository về
!git clone https://github.com/trongjhuongwr/Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update.git

# Di chuyển vào thư mục dự án
%cd Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update

Cloning into 'Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 93 (delta 36), reused 74 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 184.57 KiB | 5.43 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/kaggle/working/Deep-Learning-Based-Signature-Forgery-Detection-for-Personal-Identity-Authentication-Update


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import random
import os
from tqdm.notebook import tqdm

# Imports từ các modules của bạn
from dataloader.tSSN_trainloader import SignatureTrainDataset
from models.Triplet_Siamese_Similarity_Network import tSSN
from losses.triplet_loss import TripletLoss

print("Cài đặt và import thành công!")

Cài đặt và import thành công!


In [3]:
# --- CẤU HÌNH VÀ TẢI DỮ LIỆU ---

# Cấu hình
NUM_EPOCHS = 35  # Chỉ cần khoảng 30-50 epochs cho tiền huấn luyện
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SAVE_DIR = '/kaggle/working/pretrained_models'
os.makedirs(SAVE_DIR, exist_ok=True)

# Seed để đảm bảo kết quả có thể tái lặp
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Transform ảnh (giống như trong các notebook khác)
transform = transforms.Compose([
    transforms.Resize((220, 150)),
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Sử dụng toàn bộ dữ liệu từ tSSN_trainloader để tiền huấn luyện
# LƯU Ý: Sử dụng đường dẫn dữ liệu trên Kaggle
BASE_DATA_DIR = '/kaggle/input/cedardataset'
full_dataset = SignatureTrainDataset(
    org_dir=os.path.join(BASE_DATA_DIR, 'signatures/full_org'),
    forg_dir=os.path.join(BASE_DATA_DIR, 'signatures/full_forg'),
    transform=transform
)

train_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

print(f"Sử dụng thiết bị: {DEVICE}")
print(f"Đã tải {len(full_dataset)} triplets để tiền huấn luyện.")
print(f"Mô hình sẽ được lưu tại: {SAVE_DIR}")

Sử dụng thiết bị: cuda
Đã tải 1320 triplets để tiền huấn luyện.
Mô hình sẽ được lưu tại: /kaggle/working/pretrained_models


In [4]:
# --- KHỞI TẠO VÀ HUẤN LUYỆN ---

# Khởi tạo mô hình tSSN với backbone ResNet-34
model = tSSN(backbone_name='resnet34', output_dim=512).to(DEVICE)

# Sử dụng Triplet Loss với khoảng cách Euclidean tiêu chuẩn
loss_fn = TripletLoss(margin=0.5, mode='euclidean').to(DEVICE)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Vòng lặp huấn luyện
print("--- Bắt đầu tiền huấn luyện Feature Extractor ---")
model.train()
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")
    
    for anchor, positive, negative in progress_bar:
        anchor, positive, negative = anchor.to(DEVICE), positive.to(DEVICE), negative.to(DEVICE)
        
        optimizer.zero_grad()
        
        anchor_feat, positive_feat, negative_feat = model(anchor, positive, negative)
        
        loss = loss_fn(anchor_feat, positive_feat, negative_feat)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Average Loss: {epoch_loss:.4f}")

print("--- Hoàn thành tiền huấn luyện ---")

# --- LƯU LẠI TRỌNG SỐ CỦA FEATURE EXTRACTOR ---
feature_extractor_state_dict = model.feature_extractor.state_dict()
save_path = os.path.join(SAVE_DIR, 'pretrained_feature_extractor.pth')
torch.save(feature_extractor_state_dict, save_path)

print(f"Trọng số của Feature Extractor đã được lưu tại: {save_path}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 184MB/s]


--- Bắt đầu tiền huấn luyện Feature Extractor ---


Epoch 1/35:   0%|          | 0/42 [00:00<?, ?it/s]

Epoch 1/35 - Average Loss: 0.0463


Epoch 2/35:   0%|          | 0/42 [00:00<?, ?it/s]

...

Epoch 33/35 - Average Loss: 0.0017


Epoch 34/35:   0%|          | 0/42 [00:00<?, ?it/s]

Epoch 34/35 - Average Loss: 0.0080


Epoch 35/35:   0%|          | 0/42 [00:00<?, ?it/s]

Epoch 35/35 - Average Loss: 0.0022
--- Hoàn thành tiền huấn luyện ---
Trọng số của Feature Extractor đã được lưu tại: /kaggle/working/pretrained_models/pretrained_feature_extractor.pth
